In [1]:
import openai
import os

In [8]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [12]:
openai.api_key = os.environ.get('OPENAI_API_KEY')

In [10]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]

In [13]:
get_completion("What is 1+1?")

'1+1 equals 2.'

## Custom template

In [14]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [15]:
style = "American English in calsm and respectful tone"

In [16]:
prompt = f"""Translate the text \
that is delimited by triple backticks
into a style that is {style}.
text: ```{customer_email}```
"""
prompt

"Translate the text that is delimited by triple backticks\ninto a style that is American English in calsm and respectful tone.\ntext: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n"

In [18]:
get_completion(prompt)

"I'm really frustrated that my blender lid flew off and made a mess of my kitchen walls with smoothie! And to add to my frustration, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, my friend!"

## Usage of LangChain

In [19]:
from langchain.chat_models import ChatOpenAI

In [22]:
chat = ChatOpenAI(temperature=0)
chat

ChatOpenAI(verbose=False, callbacks=None, callback_manager=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', temperature=0.0, model_kwargs={}, openai_api_key=None, openai_api_base=None, openai_organization=None, openai_proxy=None, request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=None)

In [23]:
from langchain.prompts import ChatPromptTemplate

In [24]:
template_string = """Translate the text \
that is delimited by triple backtricks \
into a style that is {style}. \
text: ```{text}```
"""
prompt_template = ChatPromptTemplate.from_template(template_string)
prompt_template

ChatPromptTemplate(input_variables=['text', 'style'], output_parser=None, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['style', 'text'], output_parser=None, partial_variables={}, template='Translate the text that is delimited by triple backtricks into a style that is {style}. text: ```{text}```\n', template_format='f-string', validate_template=True), additional_kwargs={})])

In [27]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [29]:
customer_messages = prompt_template.format_messages(style=style, text=customer_email)

In [30]:
customer_messages

[HumanMessage(content="Translate the text that is delimited by triple backtricks into a style that is American English in calsm and respectful tone. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n", additional_kwargs={}, example=False)]

In [32]:
customer_response = chat(customer_messages)
customer_response.content

"I'm really frustrated that my blender lid flew off and made a mess of my kitchen walls with smoothie! And to make things even worse, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, my friend!"

## Reusage of prompt template

In [33]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you missues your blender \
by forgetting to put the lid on before \ 
starting a blender. \
Tough luck! See ya!
"""

In [34]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate
"""

In [35]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate, text=service_reply
)
service_messages

[HumanMessage(content="Translate the text that is delimited by triple backtricks into a style that is a polite tone that speaks in English Pirate\n. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you missues your blender by forgetting to put the lid on before \\ \nstarting a blender. Tough luck! See ya!\n```\n", additional_kwargs={}, example=False)]

In [36]:
chat(service_messages)

AIMessage(content="Ahoy there, matey! I be sorry to inform ye, but the warranty be not coverin' the expenses o' cleanin' yer galley, as 'tis yer own fault fer misusin' yer blender by forgettin' to put the lid on afore startin' it. Aye, tough luck! Farewell, me heartie!", additional_kwargs={}, example=False)

## Output Parsers usage

In [ ]:
{"gitft": False, "delivery_days": 5, "price_value": "pretty affordable"}

In [37]:
customer_review = """\
This leaf blower is pretty amaizing. It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been only one using it, and I've been \
using it every other morning to clear the leaves on our backyard.
It's slightly more expensive than the other leaf blowers \ 
out there, but I think it's worth it for the extra features.
"""

In [38]:
review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else?
delivery_days: How many days did the item take to be delivered?
price_value: Extract any sentences about the value or price.

Format the ouput as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [41]:
review_prompt_template = ChatPromptTemplate.from_template(review_template)
review_prompt_template

ChatPromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else?\ndelivery_days: How many days did the item take to be delivered?\nprice_value: Extract any sentences about the value or price.\n\nFormat the ouput as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n', template_format='f-string', validate_template=True), additional_kwargs={})])

In [43]:
messages = review_prompt_template.format_messages(text=customer_review)
chat = ChatOpenAI(temperature=0)
response = chat(messages)
response.content

'{\n  "gift": false,\n  "delivery_days": 2,\n  "price_value": "It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features."\n}'

In [44]:
print(response.content)

{
  "gift": false,
  "delivery_days": 2,
  "price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}


In [51]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

In [90]:
gift_schema = ResponseSchema(
    name="gift",
    description="Was the item purchased as a gift for someone else? Answer True if yes, False otherwise",
)
delivery_days_schema = ResponseSchema(
    name="delivery_days", description="How many days did the item take to be delivered?"
)
price_value_schema = ResponseSchema(
    name="price_value", description="Extract any sentences about the value or price."
)

response_schemas = [gift_schema, delivery_days_schema, price_value_schema]

In [91]:
output_parser = StructuredOutputParser(response_schemas=response_schemas)
format_instructions = output_parser.get_format_instructions()

print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"gift": string  // Was the item purchased as a gift for someone else? Answer True if yes, False otherwise
	"delivery_days": string  // How many days did the item take to be delivered?
	"price_value": string  // Extract any sentences about the value or price.
}
```


In [92]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else?
delivery_days: How many days did the item take to be delivered?
price_value: Extract any sentences about the value or price.

text: {text}
{format_instructions}
"""

In [93]:
prompt = ChatPromptTemplate.from_template(template=review_template_2)
messages = prompt.format_messages(
    text=customer_review, format_instructions=format_instructions
)

print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else?
delivery_days: How many days did the item take to be delivered?
price_value: Extract any sentences about the value or price.

text: This leaf blower is pretty amaizing. It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been only one using it, and I've been using it every other morning to clear the leaves on our backyard.
It's slightly more expensive than the other leaf blowers \ 
out there, but I think it's worth it for the extra features.

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"gift": string  // Was the item purchased as a gift for someone else? Answer True if yes, False otherwise
	"delivery_days

In [94]:
response = chat(messages)

print(response.content)

```json
{
	"gift": false,
	"delivery_days": "two days",
	"price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
```


In [95]:
output_dict = output_parser.parse(response.content)

output_dict

{'gift': False,
 'delivery_days': 'two days',
 'price_value': "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."}